In [1]:
import json
from config import settings
import os

# Useless outside jupiternotebook because in settings.py a line that changes the directory to src for ipynb
os.chdir(os.getcwd().replace("/src", ""))

import polars as pl
from polars import col as c
from polars import selectors as cs
import networkx as nx
from networkx_function import generate_nx_edge
from typing_extensions import Union
import numpy as np
import scipy as sp
from distflow_algorithm import DistFlow
import time
import pandapower as pp
import pandapower.networks as pn
from polars_function import (
    get_transfo_admittance,
    get_transfo_impedance,
    get_transfo_conductance,
    get_transfo_imaginary_component,
)

from data_connector import pandapower_to_distflow
from general_function import duckdb_to_dict, dict_to_duckdb, pl_to_dict
from networkx_function import get_all_edge_data, generate_shortest_path_length_matrix
import graphblas as gb
from utility.parser_utility import duckdb_to_changes_schema

## Import Schema from duckdb

In [2]:
file_names: dict[str, str] = json.load(open(settings.INPUT_FILE_NAMES))
changes_schema = duckdb_to_changes_schema(file_path=file_names["duckdb_output"])
list(changes_schema.__dict__.keys())

Read and validate tables from matlab_grid.db file: 100%|████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.05s/it]


['heartbeat',
 'resource',
 'equipment',
 'terminal',
 'busbar_section',
 'branch',
 'branch_parameter_event',
 'geo_event',
 'switch',
 'switch_event',
 'transformer',
 'transformer_end',
 'transformer_parameter_event',
 'tap',
 'tap_event',
 'bess',
 'energy_consumer',
 'external_network',
 'generating_unit',
 'container',
 'client',
 'substation',
 'base_voltage',
 'connectivity_node',
 'connectivity',
 'measurement',
 'measurement_point',
 'measurement_span']

## Filter data and create line_data

switch

branch

branch_parameter_event

In [31]:
line_data = changes_schema.resource.filter(
    (c("concrete_class") == "branch") | (c("concrete_class") == "switch")
).select(c("uuid"), c("dso_code").alias("line_id"), c("concrete_class").alias("type"))

cn_mapping = pl_to_dict(
    changes_schema.connectivity.select(
        pl.concat_str("eq_fk", "side").alias("id"), "uuid"
    )
)
## Add uuid of t1 and t2 for resource
line_data = line_data.with_columns(
    (c("uuid") + "t1")
    .replace_strict(cn_mapping, default=None)
    .alias("u_of_edge"),  # Replace eq_fk with uuid from connectivity for branch
    (c("uuid") + "t2").replace_strict(cn_mapping, default=None).alias("v_of_edge"),
)

# Add branch parameter
branch = changes_schema.branch_parameter_event.select(
    c("uuid"),
    # c("source_fk"),
    c("eq_fk"),
    c("r"),
    c("r0"),
    c("x"),
    c("x0"),
    c("b"),
    c("b0"),
    c("g"),
    c("g0"),
)
# Add branch parameter to line_data
line_data = line_data.join(branch, left_on="uuid", right_on="eq_fk", how="left").drop(
    "uuid_right"
)

In [23]:
node_id_list = (
    line_data.unpivot(on=["u_of_edge", "v_of_edge"], value_name="node_id")
    .unique("node_id", keep="first")["node_id"]
    .to_list()
)
node_id_to_nb_mapping = dict(zip(node_id_list, range(len(node_id_list))))
node_nb_to_id_mapping = dict(zip(range(len(node_id_list)), node_id_list))

line_data = line_data.with_columns(
    c(col).replace_strict(node_id_to_nb_mapping, default=None).alias(col)
    for col in ["u_of_edge", "v_of_edge"]
)
# ext_grid_nb = node_id_to_nb_mapping[ext_grid_id]

In [ ]:
# # from state_estimation_algorithm import StateEstimationData


# class StateEstimationData:
#     def __init__(self, changes_schema: dict):
#         self.line_data: pl.DataFrame
#         self.node_data: pl.DataFrame
#         self.changes_schema = changes_schema

#         self.generate_line_data()
#     ### TODO when code is ok
#     def generate_line_data(self):
#         self.line_data = self.changes_schema.resource.filter(
#             (c("concrete_class") == "branch") | (c("concrete_class") == "switch")
#         ).select(
#             c("uuid"), c("dso_code").alias("line_id"), c("concrete_class").alias("type")
#         )
# stateestimationdata = StateEstimationData(changes_schema=changes_schema)

In [8]:
ext_grid_id = "1"
n = 0.95
v_ext_grid_sq = 1.05

line_data: pl.DataFrame = pl.DataFrame(
    {
        "line_id": np.arange(1, 13).astype(str),
        "type": ["branch"] * 10 + ["transformer"] * 2,
        "u_of_edge": ["2", "2", "2", "4", "5", "6", "7", "10", "12", "13", "8", "9"],
        "v_of_edge": ["11", "1", "3", "1", "4", "4", "8", "9", "11", "11", "4", "8"],
        "n_transfo": [1.0] * 10 + [n**2, n**2],
        "x_pu": np.arange(1, 13) * 2e-3,
        "r_pu": np.arange(1, 13) * 1e-3,
        "b_pu": list(np.arange(1, 11) * -1e-3) + [0.001, 0.001],
        "g_pu": [0.0] * 10 + [0.001, 0.001],
    }
)

node_data: pl.DataFrame = pl.DataFrame(
    {
        "node_id": np.arange(1, 14).astype(str),
        "v_base": [400.0] * 7 + [200.0, 400.0, 400.0, 200.0, 100.0, 100.0],
        "p_node_pu": np.array([0, 10, 2, 0, 0, -2, 0, 7, 25, 0, 100, 0.5, 10]) * 1e-2,
        "q_node_pu": np.array([0, 0, 0.2, 2, 0, -0.2, 0, 0.7, 0, 0, 0, 0.5, 1]) * 1e-2,
    },
    strict=False,
)

np.set_printoptions(linewidth=200)
distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq

s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

numpy: 0.0014157295227050781
graphblas: 0.020789384841918945


In [9]:
s_base = 1e6
net: pp.pandapowerNet = pp.from_pickle("../data/input_grid/modified_cigre_network_lv.p")
tic = time.time()
pp.runpp(net)
print("f:", time.time() - tic)

UserWarning: File ../data/input_grid/modified_cigre_network_lv.p does not exist!!

In [4]:
ext_grid_id: str = str(net.ext_grid["bus"][0])
v_ext_grid_sq: float = net.ext_grid["vm_pu"][0] ** 2
node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)
distflow: DistFlow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)

In [5]:
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq
s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

numpy: 0.0007200241088867188
graphblas: 0.022676706314086914


In [84]:
distflow.h_gb.T

"M_150.T"            nvals  nrows  ncols  dtype         format
gb.TransposedMatrix     41     13     13   FP64  bitmapc (iso)
--------------------------------------------------------------
     0    1    2    3    4    5    6    7    8    9    10   11   12
0   1.0            1.0                      1.0  1.0  1.0          
1        1.0  1.0                                     1.0          
2             1.0                                     1.0          
3                  1.0                      1.0       1.0          
4                       1.0                 1.0       1.0          
5                  1.0       1.0            1.0       1.0          
6                                 1.0       1.0       1.0          
7        1.0  1.0                      1.0            1.0          
8                                           1.0       1.0          
9                  1.0                      1.0  1.0  1.0          
10                                                    1.0          
11            1.0                                     1.0  1.0     
12       1.0  1.0                                     1.0       1.0

In [82]:
np.multiply(s_node.reshape(-1, 1), np.random.random(100) + 0.5).transpose()

array([[0.        +0.j        , 0.95179472+0.j        , 0.09517947+0.j        , ..., 0.        +0.j        , 0.01903589+0.00190359j, 0.09517947+0.00951795j],
       [0.        +0.j        , 0.57482819+0.j        , 0.05748282+0.j        , ..., 0.        +0.j        , 0.01149656+0.00114966j, 0.05748282+0.00574828j],
       [0.        +0.j        , 0.83182895+0.j        , 0.08318289+0.j        , ..., 0.        +0.j        , 0.01663658+0.00166366j, 0.08318289+0.00831829j],
       ...,
       [0.        +0.j        , 1.41329948+0.j        , 0.14132995+0.j        , ..., 0.        +0.j        , 0.02826599+0.0028266j , 0.14132995+0.01413299j],
       [0.        +0.j        , 0.84920213+0.j        , 0.08492021+0.j        , ..., 0.        +0.j        , 0.01698404+0.0016984j , 0.08492021+0.00849202j],
       [0.        +0.j        , 0.82929242+0.j        , 0.08292924+0.j        , ..., 0.        +0.j        , 0.01658585+0.00165858j, 0.08292924+0.00829292j]])

In [77]:
s_node.reshape(-1, 1)

array([[ 0.   +0.j   ],
       [ 1.   +0.j   ],
       [ 0.1  +0.j   ],
       [ 0.07 +0.007j],
       [ 0.   +0.j   ],
       [ 0.   +0.j   ],
       [-0.02 -0.002j],
       [ 0.005+0.005j],
       [ 0.   +0.02j ],
       [ 0.25 +0.j   ],
       [ 0.   +0.j   ],
       [ 0.02 +0.002j],
       [ 0.1  +0.01j ]])

In [73]:
np.random.random(100) + 0.5

array([1.20979763, 0.7342991 , 1.17402437, 0.7730132 , 1.4262539 , 0.79595615, 1.01598385, 1.1852591 , 0.61825153, 0.9891824 , 1.15020281, 1.36583277, 1.07364581, 1.09137555, 1.11333436, 0.83503185,
       0.50154774, 0.68246054, 1.20212403, 1.23478103, 0.53457727, 1.13480669, 1.19219885, 1.43527789, 1.28531631, 1.00463642, 0.58823296, 0.56234986, 1.34667091, 1.15340563, 0.5594287 , 1.23243907,
       1.07628486, 0.62812337, 1.08384573, 0.77446393, 1.19966667, 1.28676713, 0.87357647, 1.21993883, 1.38870329, 1.42080445, 1.1968925 , 0.60623033, 0.58885734, 1.35222399, 1.19792729, 1.47520514,
       0.50217659, 1.30814651, 1.05854168, 0.5674643 , 0.91687194, 1.14234268, 0.79472783, 1.1585023 , 0.95823956, 1.15504163, 1.00367084, 1.38570815, 0.68326989, 1.44255223, 0.94303185, 1.17076271,
       1.02690779, 1.37093334, 1.48342139, 0.60668063, 1.03957436, 0.81959101, 0.65890734, 1.28476738, 1.14761778, 1.22012232, 0.68574922, 1.12463729, 1.17966652, 1.19622725, 0.70527933, 1.25037436,
     

In [6]:
line_res_pp = pl.DataFrame(
    {
        "node_id": list(net.line["to_bus"].astype(str)),
        "p_pp": list(net.res_line["p_from_mw"]),
        "q_pp": list(net.res_line["q_from_mvar"]),
    }
)

node_res_pp = pl.from_pandas(net.res_bus["vm_pu"], include_index=True).select(
    c("index").cast(pl.Utf8).alias("node_id"), "vm_pu"
)

result = (
    pl.DataFrame(
        {
            "v": np.sqrt(v),
            "p": np.real(s_flow),
            "q": np.imag(s_flow),
        }
    )
    .with_row_index(name="node_id")
    .with_columns(
        c("node_id")
        .replace_strict(distflow.node_nb_to_id_mapping, default=None)
        .alias("node_id")
    )
    .join(line_res_pp, on="node_id", how="left")
    .join(node_res_pp, on="node_id", how="left")
    .with_columns(
        ((c("v") - c("vm_pu")) / c("vm_pu")).abs().alias("diff_v"),
        ((c("p") - c("p_pp")) / c("p_pp")).abs().alias("diff_p"),
        ((c("q") - c("q_pp")) / c("p_pp")).abs().alias("diff_q"),
    )
)
print(result["diff_v", "diff_p", "diff_q"].max())

shape: (1, 3)
┌──────────┬──────────┬──────────┐
│ diff_v   ┆ diff_p   ┆ diff_q   │
│ ---      ┆ ---      ┆ ---      │
│ f64      ┆ f64      ┆ f64      │
╞══════════╪══════════╪══════════╡
│ 0.000392 ┆ 0.000182 ┆ 0.010306 │
└──────────┴──────────┴──────────┘


In [ ]:
net: pp.pandapowerNet = pp.from_pickle("../data/input_grid/synthesized_grid.p")
net.trafo["i0_percent"] = 2
tic = time.time()
pp.runpp(net)
print("f:", time.time() - tic)

f: 0.5958316326141357


In [8]:
s_base = 1e6
ext_grid_id: str = str(net.ext_grid["bus"][0])
v_ext_grid_sq: float = net.ext_grid["vm_pu"][0] ** 2

node_data, line_data = pandapower_to_distflow(net=net, s_base=s_base)
distflow = DistFlow(line_data=line_data, ext_grid_id=ext_grid_id)

In [10]:
node_data = node_data.with_columns(
    c("node_id")
    .replace_strict(distflow.node_id_to_nb_mapping, default=None)
    .alias("idx")
).sort("idx")
v0_sq = distflow.v_in_sq_np * v_ext_grid_sq
s_node = node_data["p_node_pu"].to_numpy() + 1j * node_data["q_node_pu"].to_numpy()

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(s_node=s_node, v0_sq=v0_sq, engine="numpy")
print("numpy:", time.time() - tic)

tic = time.time()
s_flow, v, i = distflow.distflow_algorithm(
    s_node=s_node, v0_sq=v0_sq, engine="graphblas"
)
print("graphblas:", time.time() - tic)

numpy: 5.96026873588562
graphblas: 0.0697178840637207


In [58]:
line_data = line_data.group_by("u_of_edge", "v_of_edge").agg(
    c("name", "n_transfo", "type").first(),
    c("r_pu", "x_pu").first() / c("r_pu", "x_pu").count(),
    c("g_pu", "b_pu").first() * c("g_pu", "b_pu").count(),
)